In [28]:
import pandas as pd
import numpy as np

In [29]:
def sigmoid(a):
    return (1/(1+np.exp(-a)))

In [30]:
def relu_der(a):
    return 1*(a>0)

In [31]:
def relu(a):
    return  a * (a > 0)

In [32]:
def forwardprop(X,W,b,activation='relu'):
    
    Z=np.dot(W,X)+b
    if(activation=='relu'):
        Z=relu(Z)
    else:
        Z=sigmoid(Z)
    A=Z
    
    goprop={'X':X,'A':A,'W':W,'Z':Z,'b':b}
    return goprop

In [33]:
def backwardprop(Y,fp,fp_old,bp,layer='mid'):
    m=Y.shape[1]
    
    if(layer=='Last'):
        dZ=fp['A']-Y
    else:
        dZ=np.dot(fp_old['W'].T,bp['dZ'])*relu_der(fp['Z'])

    dW=(1/m)*(np.dot(dZ,fp['X'].T))
    db=(1/m)*(np.sum(dZ,axis=1,keepdims=True))
    bp={'dZ':dZ,'dW':dW,'db':db}
    
    return bp

In [50]:
class NeuralNetwork:
    def __init__(self,n,layer_dims,alpha=0.1,maxiters=10):
        self.n=n
        self.layer_dims=layer_dims
        self.alpha=alpha
        self.maxiters=maxiters
        self.best_params={}
    def fit(self,X,Y):
        fps={}
        for i in range(self.n):
            fps['fp'+str(i)]={'X':X,'A':0,'W':0,'Z':0,'b':0}
        
        for i in range(self.n):
            fps['fp'+str(i)]['W']=np.random.randn(layer_dims[i+1],layer_dims[i])*0.01
            fps['fp'+str(i)]['b']=np.random.randn(layer_dims[i+1],1)
        bp={}
        fp_old={}
        for j in range(self.maxiters):
            A_prev=X
            
            for i in range(self.n):
                activation='relu'
                if(i==self.n-1):
                    activation='sigmoid'
                fps['fp'+str(i)]['X']=A_prev
                fp=forwardprop(fps['fp'+str(i)]['X'],fps['fp'+str(i)]['W'],fps['fp'+str(i)]['b'],activation)
                fps['fp'+str(i)]=fp
                A_prev=fp['A']
            for i in range(self.n-1,-1,-1):
                layer='mid'
                if(i==self.n-1):
                    layer='Last'
                bp_new=backwardprop(Y,fps['fp'+str(i)],fp_old,bp,layer)
                fps['fp'+str(i)]['W']-=self.alpha*bp_new['dW']
                fps['fp'+str(i)]['b']-=self.alpha*bp_new['db']
                fp_old=fps['fp'+str(i)]
                bp=bp_new
        self.best_params=fps
    def predict(self,X):
        X=X.T
        fps=self.best_params
        
        for i in range(self.n):
                activation='relu'
                if(i==self.n-1):
                    activation='sigmoid'
                fp=forwardprop(fps['fp'+str(i)]['X'],fps['fp'+str(i)]['W'],fps['fp'+str(i)]['b'],activation)
                fps['fp'+str(i)]=fp
        return 1*(fps['fp'+str(i)]['A']>0.5)
            

In [51]:
layer_dims=[10,5,5,4,3,1]

In [52]:
X=np.random.randn(10,10)

In [53]:
Y=np.array([1,0,1,0,1,0,1,0,1,0]).reshape(1,-1)

In [54]:
Y.shape

(1, 10)

In [55]:
nn=NeuralNetwork(5,layer_dims)

In [56]:
nn.fit(X,Y)

In [57]:
nn.predict(X)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])